In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
 76% 252M/331M [00:03<00:01, 72.2MB/s]
100% 331M/331M [00:03<00:00, 109MB/s] 


In [3]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [4]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [5]:
folder_path = '/content/UTKFace'

In [6]:
import os

age = []
gender = []
img_path = []

for file in os.listdir(folder_path):
    if file[0].isdigit():  # process only if first character is a number
        parts = file.split('_')
        age.append(int(parts[0]))
        gender.append(int(parts[1]))
        img_path.append(file)
    else:
        print(f"Skipped non-numeric filename: {file}")


In [7]:
len(age)

23708

In [8]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [9]:
df.shape

(23708, 3)

In [10]:
df.head()

,age,gender,img
0,37,1,37_1_2_20170105164301196.jpg.chip.jpg
1,45,0,45_0_0_20170120221555460.jpg.chip.jpg
2,68,0,68_0_0_20170117172618723.jpg.chip.jpg
3,61,0,61_0_2_20170111203805430.jpg.chip.jpg
4,35,1,35_1_1_20170113010415255.jpg.chip.jpg


In [11]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [12]:
train_df.shape

(20000, 3)

In [13]:
test_df.shape

(3708, 3)

In [14]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [35]:
from tensorflow.keras.utils import Sequence
import numpy as np
import cv2
import os

class MultiOutputGenerator(Sequence):
    def __init__(self, df, folder, batch_size=32, target_size=(200,200)):
        self.df = df
        self.folder = folder
        self.batch_size = batch_size
        self.target_size = target_size

    def __len__(self):
        return int(np.ceil(len(self.df)/self.batch_size))

    def __getitem__(self, idx):
        batch = self.df.iloc[idx*self.batch_size:(idx+1)*self.batch_size]
        X = np.array([cv2.resize(cv2.imread(os.path.join(self.folder,f)), self.target_size)/255.0 for f in batch['img']])
        y_age = batch['age'].values.reshape(-1,1)
        y_gender = batch['gender'].values.reshape(-1,1)
        return X, {'age': y_age, 'gender': y_gender}

train_generator = MultiOutputGenerator(train_df, folder_path)
test_generator = MultiOutputGenerator(test_df, folder_path)


In [36]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [37]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

In [38]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [39]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [40]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [41]:
model.fit(train_generator, batch_size=32, epochs=10, validation_data=test_generator)

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


625/625 ━━━━━━━━━━━━━━━━━━━━ 92s 125ms/step - age_loss: 16.1062 - age_mae: 16.1062 - gender_accuracy: 0.5192 - gender_loss: 1.6988 - loss: 184.2904 - val_age_loss: 14.1073 - val_age_mae: 14.1037 - val_gender_accuracy: 0.5216 - val_gender_loss: 0.6922 - val_loss: 82.6332
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 70s 112ms/step - age_loss: 13.8348 - age_mae: 13.8348 - gender_accuracy: 0.5258 - gender_loss: 0.6920 - loss: 82.3384 - val_age_loss: 15.8685 - val_age_mae: 15.8708 - val_gender_accuracy: 0.5216 - val_gender_loss: 0.6923 - val_loss: 84.4104
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 69s 111ms/step - age_loss: 12.9055 - age_mae: 12.9055 - gender_accuracy: 0.5211 - gender_loss: 0.6923 - loss: 81.4408 - val_age_loss: 12.3207 - val_age_mae: 12.3218 - val_gender_accuracy: 0.5216 - val_gender_loss: 0.6922 - val_loss: 80.8541
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 70s 111ms/step - age_loss: 12.1211 - age_mae: 12.1211 - gender_accuracy: 0.5235 - gender_loss: 0.6922 - loss: 80.6465 - val_age